In [4]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box
from nuscenes.utils.geometry_utils import points_in_box
from enum import IntEnum
from typing import Tuple
from pyquaternion import Quaternion
import numpy as np
import csv

In [5]:
nusc = NuScenes(version='v1.0-trainval', dataroot='/home/mike/workspaces/ECE579/data', verbose=True)

Loading NuScenes tables for version v1.0-trainval ...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 28.3 seconds.
Reverse indexing ...
Done reverse indexing in 6.5 seconds.


In [6]:
#Grab All Samples available
samples = nusc.sample[:10]

#Radar point extraction parameters
keepValidities=[0x00,0x01,0x02,0x03,0x05,0x06,0x07,0x0d,0x0e,0x04,0x08,0x09,0x0a,0x0b,0x0c,0x0f,0x10,0x11]
keepAmbigStates=range(4)

In [35]:
fieldnames = ['sample_annotation_token','category','x','y','z','dyn_prop','id','rcs','vx','vy','vx_comp','vy_comp','is_quality_valid','ambig_state','x_rms','y_rms','invalid_state','pdh0','vx_rms','vy_rms']
with open('eggs.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    #Loop over all samples avialable
    for sample in samples:
        print("New Sample")
        #Obtain all radar sample_data(SD) tokens for this sample
        FC_RadarSD = nusc.get('sample_data', sample['data']['RADAR_FRONT'])
        FL_RadarSD = nusc.get('sample_data', sample['data']['RADAR_FRONT_LEFT'])
        FR_RadarSD = nusc.get('sample_data', sample['data']['RADAR_FRONT_RIGHT'])
        RL_RadarSD = nusc.get('sample_data', sample['data']['RADAR_BACK_LEFT'])
        RR_RadarSD = nusc.get('sample_data', sample['data']['RADAR_BACK_RIGHT'])

        #GRAB Path to point clouds (pc_path), and all annotation (boxes) with this sensor data associated with them.
        #Note that boxes are transformed into sensor frame, so we need a new list of boxes for each sensor. array is only used for cameras
        FC_pc_path, boxesFC, array = nusc.get_sample_data(FC_RadarSD['token'])
        FL_pc_path, boxesFL, array = nusc.get_sample_data(FL_RadarSD['token'])
        FR_pc_path, boxesFR, array = nusc.get_sample_data(FR_RadarSD['token'])
        RL_pc_path, boxesRL, array = nusc.get_sample_data(RL_RadarSD['token'])
        RR_pc_path, boxesRR, array = nusc.get_sample_data(RR_RadarSD['token'])
        boxes = [boxesFC,boxesFL,boxesFR,boxesRL,boxesRR]

        #Obtain each radar point cloud for this sample
        FC_pc = RadarPointCloud.from_file(FC_pc_path,invalid_states=keepValidities,ambig_states=keepAmbigStates)
        FC_points = np.array(FC_pc.points[:3, :])
        FL_pc = RadarPointCloud.from_file(FL_pc_path,invalid_states=keepValidities,ambig_states=keepAmbigStates)
        FL_points = np.array(FL_pc.points[:3, :])
        FR_pc = RadarPointCloud.from_file(FR_pc_path,invalid_states=keepValidities,ambig_states=keepAmbigStates)
        FR_points = np.array(FR_pc.points[:3, :])
        RL_pc = RadarPointCloud.from_file(RL_pc_path,invalid_states=keepValidities,ambig_states=keepAmbigStates)
        RL_points = np.array(RL_pc.points[:3, :])
        RR_pc = RadarPointCloud.from_file(RR_pc_path,invalid_states=keepValidities,ambig_states=keepAmbigStates)
        RR_points = np.array(RR_pc.points[:3, :])
        #Collect all point clouds into one ndarray
        point_clouds = [FC_points,FL_points,FR_points,RL_points,RR_points]
        full_point_clouds = [FC_pc,FL_pc,FR_pc,RL_pc,RR_pc]

        sensor_num=0
        for pc in point_clouds:
            print("New Sensor")
            this_sensor_boxes = boxes[sensor_num]
            #print(this_sensor_boxes)
            this_sensor_full_pc = full_point_clouds[sensor_num]
            dims, num_points=pc.shape
            for box in this_sensor_boxes:
                box_annotation = nusc.get('sample_annotation',box.token)
                print("Num Radar Points:" + str(box_annotation['num_radar_pts']))
                if box_annotation['num_radar_pts'] == 0:
                    continue
                #print(box)
                category = box_annotation['category_name']
                point_in_box_result = points_in_box(box, pc)
                for i in range(0, num_points):
                    if point_in_box_result[i]:
                        #print(pc[:3,i])
                        writer.writerow({
                        'sample_annotation_token': box_annotation['token'],
                        'category': category,
                         'x': this_sensor_full_pc.points[0,i],
                         'y': this_sensor_full_pc.points[1,i],
                         'z': this_sensor_full_pc.points[2,i],
                         'dyn_prop': this_sensor_full_pc.points[3,i],
                         'id': this_sensor_full_pc.points[4,i],
                         'rcs': this_sensor_full_pc.points[5,i],
                         'vx': this_sensor_full_pc.points[6,i],
                         'vy': this_sensor_full_pc.points[7,i],
                         'vx_comp': this_sensor_full_pc.points[8,i],
                         'vy_comp': this_sensor_full_pc.points[9,i],
                         'is_quality_valid': this_sensor_full_pc.points[10,i],
                         'ambig_state': this_sensor_full_pc.points[11,i],
                         'x_rms': this_sensor_full_pc.points[12,i],
                         'y_rms': this_sensor_full_pc.points[13,i],
                         'invalid_state': this_sensor_full_pc.points[14,i],
                         'pdh0': this_sensor_full_pc.points[15,i],
                         'vx_rms': this_sensor_full_pc.points[16,i],
                         'vy_rms': this_sensor_full_pc.points[17,i]})
                        #print(this_sensor_full_pc.points[:,i])
            sensor_num+=1

New Sample
New Sensor
Num Radar Points:0
Num Radar Points:0
Num Radar Points:7
Num Radar Points:2
Num Radar Points:0
Num Radar Points:3
Num Radar Points:3
Num Radar Points:0
Num Radar Points:0
Num Radar Points:6
New Sensor
Num Radar Points:0
Num Radar Points:0
Num Radar Points:7
Num Radar Points:2
Num Radar Points:0
Num Radar Points:3
Num Radar Points:3
Num Radar Points:0
Num Radar Points:0
Num Radar Points:6
New Sensor
Num Radar Points:0
Num Radar Points:0
Num Radar Points:7
Num Radar Points:2
Num Radar Points:0
Num Radar Points:3
Num Radar Points:3
Num Radar Points:0
Num Radar Points:0
Num Radar Points:6
New Sensor
Num Radar Points:0
Num Radar Points:0
Num Radar Points:7
Num Radar Points:2
Num Radar Points:0
Num Radar Points:3
Num Radar Points:3
Num Radar Points:0
Num Radar Points:0
Num Radar Points:6
New Sensor
Num Radar Points:0
Num Radar Points:0
Num Radar Points:7
Num Radar Points:2
Num Radar Points:0
Num Radar Points:3
Num Radar Points:3
Num Radar Points:0
Num Radar Points:0
Num

New Sensor
Num Radar Points:0
Num Radar Points:0
Num Radar Points:0
Num Radar Points:7
Num Radar Points:0
Num Radar Points:7
Num Radar Points:0
Num Radar Points:0
Num Radar Points:9
Num Radar Points:5
Num Radar Points:0
Num Radar Points:0
Num Radar Points:2
Num Radar Points:10
Num Radar Points:0
Num Radar Points:0
New Sensor
Num Radar Points:0
Num Radar Points:0
Num Radar Points:0
Num Radar Points:7
Num Radar Points:0
Num Radar Points:7
Num Radar Points:0
Num Radar Points:0
Num Radar Points:9
Num Radar Points:5
Num Radar Points:0
Num Radar Points:0
Num Radar Points:2
Num Radar Points:10
Num Radar Points:0
Num Radar Points:0
New Sensor
Num Radar Points:0
Num Radar Points:0
Num Radar Points:0
Num Radar Points:7
Num Radar Points:0
Num Radar Points:7
Num Radar Points:0
Num Radar Points:0
Num Radar Points:9
Num Radar Points:5
Num Radar Points:0
Num Radar Points:0
Num Radar Points:2
Num Radar Points:10
Num Radar Points:0
Num Radar Points:0
New Sensor
Num Radar Points:0
Num Radar Points:0
Num

In [17]:
box_annotation

{'token': 'b0284e14d17a444a8d0071bd1f03a0a2',
 'sample_token': 'e93e98b63d3b40209056d129dc53ceee',
 'instance_token': 'a1f53dd65da64eb2b0cec51d57b8c047',
 'visibility_token': '4',
 'attribute_tokens': ['58aa28b1c2a54dc88e169808c07331e3'],
 'translation': [994.117, 639.415, 1.981],
 'size': [2.156, 6.227, 2.601],
 'rotation': [0.9503529315429128, 0.0, 0.0, -0.3111740758928862],
 'prev': '',
 'next': '405f156885d549ca812e1305eddee42f',
 'num_lidar_pts': 42,
 'num_radar_pts': 6,
 'category_name': 'vehicle.truck'}

In [3]:
def intersecting_points(points, boxes, ):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-3-c18552893530>, line 2)

In [ ]:
   #For each sample, loop over each sample_annotation
    for annotationToken in sample['anns']

annotation = nusc.get('sample_annotation',annotationToken)
        num_radar_points=annotation['num_radar_pts']
        
        #Make sure the annotation includes radar points, otherwise skip it
        if num_radar_points == 0:
            continue